<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_0618/blob/main/tibame_practice_momo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install dirtyjson

In [24]:
import urllib.request as req
import bs4 as bs
import json
import dirtyjson

url = "https://www.momoshop.com.tw/search/searchShop.jsp?keyword=%E5%90%B9%E9%A2%A8%E6%A9%9F&_isFuzzy=0&searchType=1&cateLevel=0"
resp = req.urlopen(url)
html = bs.BeautifulSoup(resp.read())

scripts = html.find_all("script")
target_script = None
for script in scripts:
    if "@type" in script.text:
        target_script = script
        break
datas = dirtyjson.loads(target_script.text)
f = open("data.json", "w", encoding="utf-8")
json.dump(datas, f, indent=4, ensure_ascii=False)
f.close()

In [26]:
import urllib.request as req
import bs4 as bs
import json
import dirtyjson

url = "https://www.momoshop.com.tw/search/searchShop.jsp?keyword=%E5%90%B9%E9%A2%A8%E6%A9%9F&_isFuzzy=0&searchType=1&cateLevel=0"
resp = req.urlopen(url)
html = bs.BeautifulSoup(resp.read())

scripts = html.find_all("script")
target_script = None
for script in scripts:
    if "@type" in script.text:
        target_script = script
        break
datas = dirtyjson.loads(target_script.text)
items = datas["mainEntity"]["itemListElement"]
for item in items:
    print(item["name"])

【KINYO】BLDC高速無刷負離子吹風機KH-9688(11萬轉速2億負離子)
【dyson 戴森】HD08 Origin Supersonic 吹風機 溫控 負離子(瑰麗紅色 平裝版)
【小不記】可折疊高速負離子吹風機 強勁大風量(智能控溫護髮 速乾 附吹風機收納架)
【KINYO】全球電壓無刷吹風機(全球電壓/四億負離子 KH-9660)
【TECO 東元】BLDC專業負離子吹風機(XYFXZ003)
【SHARP 夏普】四氣流水潤溫控吹風機-午夜黑(IB-WX901T-B)
【Panasonic 國際牌】奈米水離子吹風機(EH-NA27-W)
【MIDORI 美多莉】Hypersonic高速負離子吹風機-霧夜金(全配組-不含收納架)
【KINYO】全球電壓無刷吹風機(全球電壓/四億負離子 KH-9660)
【Panasonic 國際牌】高速馬達負離子吹風機(EH-NE6M)
【Panasonic 國際牌】高速馬達吹風機-奶茶(EH-NE6M-C)
【小米官方旗艦館】Xiaomi 負離子高速吹風機
【Panasonic 國際牌】奈米水離子吹風機(EH-NA27-PP)
【Dreame 追覓科技】Pocket Uni 魔術折疊高速吹風機(100-240V國際電壓版)
【Panasonic 國際牌】奈米水離子吹風機-搭載捲髮定型烘罩(EH-NA9L-RP)
【TESCOM】大風量負離子吹風機(TID450TW)
【SOLAC】專業負離子吹風機  白/紫/灰/粉 色(SD-1000)
【TESCOM】BLDC輕巧負離子吹風機(TD760TW)
【TESCOM】大風量負離子吹風機(TID292TW)
【Panasonic 國際牌】奈米水離子吹風機-銀(EH-NA7M-H)
【dyson 戴森】HD15 Supersonic 吹風機 溫控 負離子(普魯士藍禮盒版)
【TESCOM】(福利品微盒損)防靜電負離子大風量渦流吹風機 TID2200TW(-珍珠白)
【Philips 飛利浦】官方直營 沙龍級護髮負離子吹風機-霧藍黑(BHD518/01)
【TESCOM】專業型大風量負離子吹風機(TID960TW)
【小米官方旗艦館】小米負離子吹風機 H300 白色
【Panasonic 國際牌】奈米水離子吹風機-灰(EH-NA9N-H)
【Dreame 追覓科技】Pocket 